In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.5 MB/s eta 0:00:00


In [ ]:
# 1. Mount Drive
from google.colab import drive
drive.mount('/content/drive')

# 2. Install Dependencies
!pip install torch torch-geometric networkx scikit-learn

# 3. Load Dataset
import torch
import os

file_path = '/content/drive/My Drive/airport_gnn_dataset.pt'

if not os.path.exists(file_path):
    print(f"❌ Error: File not found at {file_path}")
    print("Please upload the NEW 'airport_gnn_dataset.pt' to your Drive root.")
else:
    # weights_only=False is required for custom objects
    checkpoint = torch.load(file_path, weights_only=False)

    dataset = checkpoint['dataset']
    encoders = checkpoint['encoders']

    print(f"✅ Successfully loaded {len(dataset)} scenarios.")

    # Verify the new fields exist
    sample = dataset[0]
    if hasattr(sample, 'flow_jitter'):
        print(f"✅ Dataset Verification: Jitter and Throughput fields found.")
    else:
        print(f"⚠️ WARNING: Dataset missing 'flow_jitter'. You might be using the old .pt file.")

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.9 MB/s eta 0:00:00
✅ Successfully loaded 1000 scenarios.
✅ Dataset Verification: Jitter and Throughput fields found.


/usr/local/lib/python3.12/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.7.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATv2Conv

class ZoneAwareGNN(nn.Module):
    def __init__(self, node_feature_dim=32, edge_feature_dim=16, flow_feature_dim=24, hidden_dim=128, num_heads=4):
        super(ZoneAwareGNN, self).__init__()

        # --- Encoders ---
        self.node_encoder = nn.Sequential(
            nn.Linear(node_feature_dim, hidden_dim), nn.ReLU(), nn.Linear(hidden_dim, hidden_dim)
        )
        self.edge_encoder = nn.Sequential(
            nn.Linear(edge_feature_dim, hidden_dim // 2), nn.ReLU()
        )
        self.flow_encoder = nn.Sequential(
            nn.Linear(flow_feature_dim, hidden_dim), nn.ReLU()
        )

        # --- Graph Core (GATv2) ---
        self.gat1 = GATv2Conv(hidden_dim, hidden_dim, heads=num_heads, edge_dim=hidden_dim//2, concat=True)
        self.gat2 = GATv2Conv(hidden_dim*num_heads, hidden_dim, heads=1, edge_dim=hidden_dim//2, concat=False)

        # --- Prediction Heads ---
        context_dim = hidden_dim * 3

        self.head_latency = nn.Sequential(
            nn.Linear(context_dim, hidden_dim), nn.ReLU(), nn.Linear(hidden_dim, 1)
        )
        self.head_jitter = nn.Sequential(
            nn.Linear(context_dim, hidden_dim), nn.ReLU(), nn.Linear(hidden_dim, 1)
        )
        # [FIX] Loss is now REGRESSION (0.0 to 1.0)
        # We Add Sigmoid here so the model naturally predicts a percentage
        self.head_loss = nn.Sequential(
            nn.Linear(context_dim, hidden_dim), nn.ReLU(), nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )
        self.head_throughput = nn.Sequential(
            nn.Linear(context_dim, hidden_dim), nn.ReLU(), nn.Linear(hidden_dim, 1)
        )

    def forward(self, data):
        x = self.node_encoder(data.x)
        edge_attr = self.edge_encoder(data.edge_attr)
        x = F.elu(self.gat1(x, data.edge_index, edge_attr))
        node_embeddings = self.gat2(x, data.edge_index, edge_attr)

        flow_emb = self.flow_encoder(data.flow_features)
        src_ctx = node_embeddings[data.flow_to_node[:, 0]]
        dst_ctx = node_embeddings[data.flow_to_node[:, 1]]
        combined = torch.cat([flow_emb, src_ctx, dst_ctx], dim=1)

        return {
            'latency': self.head_latency(combined).squeeze(-1),
            'jitter': self.head_jitter(combined).squeeze(-1),
            'loss': self.head_loss(combined).squeeze(-1),
            'throughput': self.head_throughput(combined).squeeze(-1)
        }

In [ ]:
# --- CELL 3: REGRESSION TRAINING ---
from torch_geometric.loader import DataLoader
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np

# 1. Setup Data
clean_dataset = [d for d in dataset if d is not None]
train_size = int(0.8 * len(clean_dataset))
test_size = len(clean_dataset) - train_size
train_data, test_data = torch.utils.data.random_split(clean_dataset, [train_size, test_size])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 2. Setup Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🚀 Training on: {device}")

model = ZoneAwareGNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# --- HYPERPARAMETERS ---
SCALE_JITTER = 100.0
# [FIX] No more Class Weighting. We treat Loss as a physics problem (MSE).
criterion_loss = nn.MSELoss()

epochs = 30

print(f"{'Epoch':<6} | {'Train Loss':<10} | {'Test Loss':<10} | {'Lat Err':<10} | {'Loss Err':<10}")
print("-" * 65)

for epoch in range(epochs):
    # --- TRAIN ---
    model.train()
    train_loss_accum = 0

    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch)

        l_lat = F.mse_loss(out['latency'], torch.log1p(batch.flow_latency))
        l_jit = F.mse_loss(out['jitter'], batch.flow_jitter * SCALE_JITTER)
        # [FIX] MSE Loss for Packet Loss (Regression)
        l_pkt = criterion_loss(out['loss'], batch.flow_packet_loss)
        l_tput = F.mse_loss(out['throughput'], batch.flow_throughput)

        # Equal weights because all are now Regression
        loss = l_lat + l_jit + (l_pkt * 5.0) + (l_tput * 0.1)
        loss.backward()
        optimizer.step()
        train_loss_accum += loss.item()

    avg_train_loss = train_loss_accum / len(train_loader)

    # --- TEST ---
    model.eval()
    test_loss_accum = 0
    total_lat_err = 0
    total_loss_err = 0

    with torch.no_grad():
        for batch in test_loader:
            batch = batch.to(device)
            out = model(batch)

            l_lat = F.mse_loss(out['latency'], torch.log1p(batch.flow_latency))
            l_jit = F.mse_loss(out['jitter'], batch.flow_jitter * SCALE_JITTER)
            l_pkt = criterion_loss(out['loss'], batch.flow_packet_loss)
            l_tput = F.mse_loss(out['throughput'], batch.flow_throughput)
            loss = l_lat + l_jit + (l_pkt * 5.0) + (l_tput * 0.1)
            test_loss_accum += loss.item()

            # Metrics
            pred_lat = torch.expm1(out['latency'])
            lat_err = torch.abs(pred_lat - batch.flow_latency).mean().item()
            total_lat_err += lat_err

            # Packet Loss Error (Absolute Diff)
            loss_err = torch.abs(out['loss'] - batch.flow_packet_loss).mean().item()
            total_loss_err += loss_err

    avg_test_loss = test_loss_accum / len(test_loader)
    avg_lat_err = total_lat_err / len(test_loader)
    avg_loss_err = total_loss_err / len(test_loader)

    if (epoch + 1) % 5 == 0:
        print(f"{epoch+1:<6} | {avg_train_loss:<10.2f} | {avg_test_loss:<10.2f} | {avg_lat_err:<10.4f} | {avg_loss_err:<10.4f}")

print("✅ Training Complete.")

🚀 Training on: cuda
Epoch  | Train Loss | Test Loss  | Lat Err    | Loss Err  
-----------------------------------------------------------------
5      | 231.32     | 214.81     | 5.8599     | 0.1797    
10     | 229.27     | 216.37     | 7.6458     | 0.1905    
15     | 227.49     | 211.93     | 4.5106     | 0.1697    
20     | 224.47     | 205.53     | 4.6348     | 0.1736    
25     | 217.37     | 200.90     | 3.5786     | 0.1737    
30     | 214.38     | 200.00     | 3.2860     | 0.1720    
✅ Training Complete.


In [ ]:
# --- CELL 4: FINAL REPORT ---
import numpy as np
import pandas as pd
import torch

model.eval()
results = []

print("Running inference...")

with torch.no_grad():
    for batch in test_loader:
        batch = batch.to(device)
        out = model(batch)

        pred_lat = torch.expm1(out['latency']).cpu().numpy()
        true_lat = batch.flow_latency.cpu().numpy()

        # Jitter: Divide by 100
        pred_jit = (out['jitter'] / 100.0).cpu().numpy()
        true_jit = batch.flow_jitter.cpu().numpy()

        # Packet Loss: RAW OUTPUT (Sigmoid is already in Model)
        pred_loss = out['loss'].cpu().numpy()
        true_loss = batch.flow_packet_loss.cpu().numpy()

        pred_tput = out['throughput'].cpu().numpy()
        true_tput = batch.flow_throughput.cpu().numpy()

        locs = batch.flow_location.cpu().numpy()

        for i in range(len(pred_lat)):
            results.append({
                'Location_Digit': locs[i],
                'Lat_Pred': pred_lat[i], 'Lat_True': true_lat[i],
                'Jitter_Pred': pred_jit[i], 'Jitter_True': true_jit[i],
                'Loss_Pred': pred_loss[i], 'Loss_True': true_loss[i],
                'Tput_Pred': pred_tput[i], 'Tput_True': true_tput[i]
            })

df = pd.DataFrame(results)
df['Location'] = encoders['location'].inverse_transform(df['Location_Digit'])

summary = df.groupby('Location').agg({
    'Lat_Pred': 'mean', 'Lat_True': 'mean',
    'Jitter_Pred': 'mean', 'Jitter_True': 'mean',
    'Loss_Pred': 'mean', 'Loss_True': 'mean',
    'Tput_Pred': 'mean', 'Tput_True': 'mean'
}).reset_index()

print("\n--- 🌍 Digital Twin Location Report (Regression Mode) ---")
display(summary.sort_values('Loss_Pred', ascending=False).head(15))

Running inference...

--- 🌍 Digital Twin Location Report (Regression Mode) ---


,Location,Lat_Pred,Lat_True,Jitter_Pred,Jitter_True,Loss_Pred,Loss_True,Tput_Pred,Tput_True
2,Retail/Shops,13.597823,13.703500,0.066524,0.079945,0.230320,0.200584,4.771941,4.891645
7,TerminalA/VIP,13.556322,13.533334,0.013985,0.011065,0.226214,0.201824,8.006184,6.512033
6,TerminalA/Gates_A,13.312809,13.602500,0.017013,0.015428,0.219044,0.132776,8.323447,7.075086
10,TerminalB/VIP,11.751876,12.915000,0.100982,0.078499,0.212081,0.210806,5.311388,6.442760
5,TerminalA/CheckIn,12.060356,13.521875,0.083252,0.100358,0.210958,0.144505,4.626361,5.234698
13,TerminalC/Gates_C,12.919518,13.223125,0.063463,0.060851,0.210089,0.145277,6.164370,7.015298
11,TerminalC/Arrival,12.737844,13.560000,0.072891,0.088714,0.209144,0.211221,5.122427,4.826694
9,TerminalB/Gates_B,11.935235,12.790417,0.107191,0.111830,0.207584,0.163722,6.325953,7.020214
12,TerminalC/Baggage,12.594273,13.586667,0.072068,0.068593,0.206821,0.156348,5.144037,5.162239
8,TerminalB/CheckIn,11.974046,13.476250,0.122378,0.141017,0.204401,0.161416,4.533520,5.150564


In [ ]:
# --- CELL 5: FINAL VERIFICATION (Renamed) ---
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, mean_absolute_error

model.eval()

# Storage
true_lat, pred_lat = [], []
true_jit, pred_jit = [], []
true_loss, pred_loss = [], []
true_tput, pred_tput = [], []

print("Calculating Final Metrics...")

with torch.no_grad():
    for batch in test_loader:
        batch = batch.to(device)
        out = model(batch)

        # 1. Latency (Un-log)
        true_lat.extend(batch.flow_latency.cpu().numpy())
        pred_lat.extend(torch.expm1(out['latency']).cpu().numpy())

        # 2. Jitter (Divide by 100)
        true_jit.extend(batch.flow_jitter.cpu().numpy())
        pred_jit.extend((out['jitter'] / 100.0).cpu().numpy())

        # 3. Packet Loss (Regression Output 0.0-1.0)
        true_loss.extend(batch.flow_packet_loss.cpu().numpy())
        pred_loss.extend(out['loss'].cpu().numpy())

        # 4. Throughput
        true_tput.extend(batch.flow_throughput.cpu().numpy())
        pred_tput.extend(out['throughput'].cpu().numpy())

# --- METRIC CALCULATIONS ---
def calculate_wape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    y_pred = np.maximum(y_pred, 0) # Clip negatives
    return (np.sum(np.abs(y_true - y_pred)) / (np.sum(np.abs(y_true)) + 1e-6)) * 100

# 1. Regression Metrics (Percentage)
wape_lat = calculate_wape(true_lat, pred_lat)
wape_tput = calculate_wape(true_tput, pred_tput)

# 2. Physical Metrics (Absolute Error)
mae_jit = mean_absolute_error(true_jit, pred_jit)
mae_loss = mean_absolute_error(true_loss, pred_loss)

# 3. Critical Event Detection (Renamed from "Storm")
# We classify a flow as "Congested" if packet loss > 15%
CONGESTION_THRESHOLD = 0.15
binary_preds = [1 if p > CONGESTION_THRESHOLD else 0 for p in pred_loss]
binary_true = [1 if t > CONGESTION_THRESHOLD else 0 for t in true_loss]

acc_cong = accuracy_score(binary_true, binary_preds) * 100
prec_cong = precision_score(binary_true, binary_preds, zero_division=0) * 100
rec_cong = recall_score(binary_true, binary_preds, zero_division=0) * 100
conf_matrix = confusion_matrix(binary_true, binary_preds)

# --- REPORT ---
print("\n" + "="*50)
print(f"🌍 DIGITAL TWIN PERFORMANCE REPORT")
print("="*50)

print(f"⏱️  LATENCY (ms)")
print(f"   - Prediction Accuracy: {max(0, 100 - wape_lat):.2f}%")
print(f"   - Error Margin:        {wape_lat:.2f}%")
print("-" * 50)

print(f"📉  JITTER (ms)")
print(f"   - Avg Physical Error:  {mae_jit:.5f} ms")
print("-" * 50)

print(f"📶  THROUGHPUT (Mbps)")
print(f"   - Prediction Accuracy: {max(0, 100 - wape_tput):.2f}%")
print(f"   - Error Margin:        {wape_tput:.2f}%")
print("-" * 50)

print(f"🔻  PACKET LOSS RATE (0.0 - 1.0)")
print(f"   - Avg Absolute Error:  {mae_loss:.4f} ({mae_loss*100:.1f}%)")
print("-" * 50)

print(f"🚨  CRITICAL CONGESTION DETECTION (Loss > {CONGESTION_THRESHOLD*100}%)")
print(f"   - Classification Acc:  {acc_cong:.2f}%")
print(f"   - Precision:           {prec_cong:.2f}% (Valid Alerts)")
print(f"   - Recall:              {rec_cong:.2f}% (Congestion Caught)")
print(f"   - Confusion Matrix:\n{conf_matrix}")
print("     [Normal->Normal,   Normal->Congested]")
print("     [Congested->Normal, Congested->Congested]")
print("="*50)

Calculating Final Metrics...

🌍 DIGITAL TWIN PERFORMANCE REPORT
⏱️  LATENCY (ms)
   - Prediction Accuracy: 76.64%
   - Error Margin:        23.36%
--------------------------------------------------
📉  JITTER (ms)
   - Avg Physical Error:  0.04611 ms
--------------------------------------------------
📶  THROUGHPUT (Mbps)
   - Prediction Accuracy: 79.27%
   - Error Margin:        20.73%
--------------------------------------------------
🔻  PACKET LOSS RATE (0.0 - 1.0)
   - Avg Absolute Error:  0.1662 (16.6%)
--------------------------------------------------
🚨  CRITICAL CONGESTION DETECTION (Loss > 15.0%)
   - Classification Acc:  78.76%
   - Precision:           47.02% (Valid Alerts)
   - Recall:              100.00% (Congestion Caught)
   - Confusion Matrix:
[[15217  5395]
 [    0  4788]]
     [Normal->Normal,   Normal->Congested]
     [Congested->Normal, Congested->Congested]


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.scatter(df['Actual_Latency'], df['Predicted_Latency'], alpha=0.3, s=10)
plt.plot([0, df['Actual_Latency'].max()], [0, df['Actual_Latency'].max()], 'r--', label='Perfect Prediction')

plt.xlabel('Ground Truth Latency (ms)')
plt.ylabel('GNN Predicted Latency (ms)')
plt.title('Digital Twin Accuracy: Predicted vs Actual Flow Latency')
plt.legend()
plt.grid(True)
plt.show()